In [1]:
import warnings

warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np

from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin

from mypipes import *

In [2]:
train_file=r"E:\IIT-K\data\data\loan_data_train.csv"
test_file=r"E:\IIT-K\data\data\loan_data_test.csv"

#train_file=r"D:\IIT-K\data\data\loan_data_train.csv"
#test_file=r"D:\IIT-K\data\data\loan_data_test.csv"

ld_train=pd.read_csv(train_file)
ld_test=pd.read_csv(test_file)

ld_train.head()


,ID,Amount.Requested,Amount.Funded.By.Investors,Interest.Rate,Loan.Length,Loan.Purpose,Debt.To.Income.Ratio,State,Home.Ownership,Monthly.Income,FICO.Range,Open.CREDIT.Lines,Revolving.CREDIT.Balance,Inquiries.in.the.Last.6.Months,Employment.Length
0,79542.0,25000,25000,18.49%,60 months,debt_consolidation,27.56%,VA,MORTGAGE,8606.56,720-724,11,15210,3.0,5 years
1,75473.0,19750,19750,17.27%,60 months,debt_consolidation,13.39%,NY,MORTGAGE,6737.50,710-714,14,19070,3.0,4 years
2,67265.0,2100,2100,14.33%,36 months,major_purchase,3.50%,LA,OWN,1000.00,690-694,13,893,1.0,< 1 year
3,80167.0,28000,28000,16.29%,36 months,credit_card,19.62%,NV,MORTGAGE,7083.33,710-714,12,38194,1.0,10+ years
4,17240.0,24250,17431.82,12.23%,60 months,credit_card,23.79%,OH,MORTGAGE,5833.33,730-734,6,31061,2.0,10+ years


In [3]:
ld_test.head()

,ID,Amount.Requested,Amount.Funded.By.Investors,Loan.Length,Loan.Purpose,Debt.To.Income.Ratio,State,Home.Ownership,Monthly.Income,FICO.Range,Open.CREDIT.Lines,Revolving.CREDIT.Balance,Inquiries.in.the.Last.6.Months,Employment.Length
0,20093,5000,5000,60 months,moving,12.59%,NY,RENT,4416.67,690-694,13,7686,0,< 1 year
1,62445,18000,18000,60 months,debt_consolidation,4.93%,CA,RENT,5258.50,710-714,6,11596,0,10+ years
2,65248,7200,7200,60 months,debt_consolidation,25.16%,LA,MORTGAGE,3750.00,750-754,13,7283,0,6 years
3,81822,7200,7200,36 months,debt_consolidation,17.27%,NY,MORTGAGE,3416.67,790-794,14,4838,0,10+ years
4,57923,22000,22000,60 months,debt_consolidation,18.28%,MI,MORTGAGE,6083.33,720-724,9,20181,0,8 years


In [4]:
ld_train.sample(10)

,ID,Amount.Requested,Amount.Funded.By.Investors,Interest.Rate,Loan.Length,Loan.Purpose,Debt.To.Income.Ratio,State,Home.Ownership,Monthly.Income,FICO.Range,Open.CREDIT.Lines,Revolving.CREDIT.Balance,Inquiries.in.the.Last.6.Months,Employment.Length
1935,8378.0,7500,7475,10.99%,36 months,home_improvement,7.51%,TX,MORTGAGE,7083.33,800-804,9,2245,7.0,8 years
1513,39867.0,10000,10000,11.71%,36 months,other,13.41%,NJ,RENT,5800.00,710-714,9,4214,0.0,3 years
550,54692.0,9600,9600,10.74%,60 months,other,3.58%,MI,MORTGAGE,8000.00,725-729,7,3207,0.0,3 years
1799,45094.0,12000,12000,17.99%,36 months,other,19.89%,CA,RENT,5129.00,660-664,8,8923,0.0,5 years
1632,78856.0,8325,8325,17.77%,36 months,debt_consolidation,20.75%,MO,RENT,2000.00,665-669,7,7676,0.0,< 1 year
741,80125.0,12000,11975,12.12%,36 months,debt_consolidation,10.84%,NJ,RENT,7083.33,685-689,11,20601,0.0,4 years
608,74944.0,20950,20875,21.49%,60 months,debt_consolidation,27.56%,NY,MORTGAGE,4500.00,675-679,15,24210,1.0,6 years
1443,83786.0,6000,6000,15.80%,36 months,medical,7.94%,NY,RENT,5000.00,670-674,7,14784,1.0,10+ years
1079,29794.0,4800,4800,10.59%,36 months,debt_consolidation,20.21%,IL,RENT,2666.67,710-714,5,3831,1.0,6 years
900,37041.0,30000,29950,7.90%,36 months,home_improvement,0.34%,MA,RENT,13416.67,810-814,9,2174,3.0,10+ years


In [5]:
ld_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2200 entries, 0 to 2199
Data columns (total 15 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   ID                              2199 non-null   float64
 1   Amount.Requested                2199 non-null   object 
 2   Amount.Funded.By.Investors      2199 non-null   object 
 3   Interest.Rate                   2200 non-null   object 
 4   Loan.Length                     2199 non-null   object 
 5   Loan.Purpose                    2199 non-null   object 
 6   Debt.To.Income.Ratio            2199 non-null   object 
 7   State                           2199 non-null   object 
 8   Home.Ownership                  2199 non-null   object 
 9   Monthly.Income                  2197 non-null   float64
 10  FICO.Range                      2200 non-null   object 
 11  Open.CREDIT.Lines               2196 non-null   object 
 12  Revolving.CREDIT.Balance        21

In [6]:
ld_train["Loan.Length"].value_counts(dropna=False)

36 months    1722
60 months     476
NaN             1
.               1
Name: Loan.Length, dtype: int64

In [7]:
#PYTHON PIPELINE
# dont need : ID , Intereste.Rate

# Amount requested : V1
# Amount.Funded.By.Investors : V2 
# Open.CREDIT.Lines : V3
# Revolving.CREDIT.Balance : V4

# convert it to numeric , and then impute missing values with median

# Debt.To.Income.Ratio : V5

# remove percentage sign , covert to numeric and then impute with median

# Loan.Length ,Loan.Purpose,State,Home.Ownership,Employment.Length
# V6-V10
# create dummies for these, considering frequency cutoff as 20

# Monthly.Income : V11,
# Inquiries.in.the.Last.6.Months :V12

# impute missing values with median

# FICO.Range : V13

# split the column at '-', convert the resulting columns to numeric(say a ,b) , 
# then create new column = (a+b)/2 


In [8]:
# pipe_name=pdPipeline([
#     ('name of the process',call to process),
#     ('name of the process', call to process),
#     .....
# ])

In [9]:
p1=pdPipeline([
    ('var_select',VarSelector(['Amount.Requested','Amount.Funded.By.Investors',
                               'Open.CREDIT.Lines','Revolving.CREDIT.Balance'])),
    ('convert_to_numeric',convert_to_numeric()),
    ('missing_trt',DataFrameImputer())
])

In [10]:
p2=pdPipeline([
    ('var_select',VarSelector(['Debt.To.Income.Ratio'])),
    ('string_clean',string_clean(replace_it='%',replace_with='')),
    ('convert_to_numeric',convert_to_numeric()),
    ('missing_trt',DataFrameImputer())
])

In [11]:
p3=pdPipeline([
    ('var_select',VarSelector(['Loan.Length', 'Loan.Purpose','State','Home.Ownership',
                               'Employment.Length'])),
    ('missing_trt',DataFrameImputer()),
    ('create_dummies',get_dummies_Pipe(20))
])

In [12]:
p4=pdPipeline([
    ('var_select',VarSelector(['Monthly.Income','Inquiries.in.the.Last.6.Months'])),
    ('missing_trt',DataFrameImputer())
])

In [13]:
p5=pdPipeline([
    ('var_select',VarSelector(['FICO.Range'])),
    ('custom_fico',custom_fico()),
    ('missing_trt',DataFrameImputer())
])


In [14]:
data_pipe=FeatureUnion([
    ('obj_to_num',p1),
    ('dtir',p2),
    ('obj_to_dum',p3),
    ('num',p4),
    ('fico',p5)
])

In [15]:
data_pipe.fit(ld_train)

FeatureUnion(transformer_list=[('obj_to_num',
                                pdPipeline(steps=[('var_select',
                                                   VarSelector(feature_names=['Amount.Requested',
                                                                              'Amount.Funded.By.Investors',
                                                                              'Open.CREDIT.Lines',
                                                                              'Revolving.CREDIT.Balance'])),
                                                  ('convert_to_numeric',
                                                   convert_to_numeric()),
                                                  ('missing_trt',
                                                   DataFrameImputer())])),
                               ('dtir',
                                pdPipeline(steps=[('var_select',
                                                   VarSelector(feature_names=...
 

In [16]:
len(data_pipe.get_feature_names())

61

In [17]:
data_pipe.transform(ld_train).shape

(2200, 61)

In [18]:
x_train=pd.DataFrame(data=data_pipe.transform(ld_train),
                    columns=data_pipe.get_feature_names())

In [19]:
x_test=pd.DataFrame(data=data_pipe.transform(ld_test),
                    columns=data_pipe.get_feature_names())

In [20]:
x_train.shape

(2200, 61)

In [21]:
x_test.shape

(300, 61)

In [22]:
x_train.columns

Index(['obj_to_num__Amount.Requested',
       'obj_to_num__Amount.Funded.By.Investors',
       'obj_to_num__Open.CREDIT.Lines', 'obj_to_num__Revolving.CREDIT.Balance',
       'dtir__Debt.To.Income.Ratio', 'obj_to_dum__Loan.Length_36 months',
       'obj_to_dum__Loan.Length_60 months',
       'obj_to_dum__Loan.Purpose_debt_consolidation',
       'obj_to_dum__Loan.Purpose_credit_card',
       'obj_to_dum__Loan.Purpose_other',
       'obj_to_dum__Loan.Purpose_home_improvement',
       'obj_to_dum__Loan.Purpose_major_purchase',
       'obj_to_dum__Loan.Purpose_small_business',
       'obj_to_dum__Loan.Purpose_car', 'obj_to_dum__Loan.Purpose_wedding',
       'obj_to_dum__Loan.Purpose_medical', 'obj_to_dum__Loan.Purpose_moving',
       'obj_to_dum__State_CA', 'obj_to_dum__State_NY', 'obj_to_dum__State_FL',
       'obj_to_dum__State_TX', 'obj_to_dum__State_PA', 'obj_to_dum__State_IL',
       'obj_to_dum__State_GA', 'obj_to_dum__State_NJ', 'obj_to_dum__State_VA',
       'obj_to_dum__State_MA',

In [23]:
x_test.columns

Index(['obj_to_num__Amount.Requested',
       'obj_to_num__Amount.Funded.By.Investors',
       'obj_to_num__Open.CREDIT.Lines', 'obj_to_num__Revolving.CREDIT.Balance',
       'dtir__Debt.To.Income.Ratio', 'obj_to_dum__Loan.Length_36 months',
       'obj_to_dum__Loan.Length_60 months',
       'obj_to_dum__Loan.Purpose_debt_consolidation',
       'obj_to_dum__Loan.Purpose_credit_card',
       'obj_to_dum__Loan.Purpose_other',
       'obj_to_dum__Loan.Purpose_home_improvement',
       'obj_to_dum__Loan.Purpose_major_purchase',
       'obj_to_dum__Loan.Purpose_small_business',
       'obj_to_dum__Loan.Purpose_car', 'obj_to_dum__Loan.Purpose_wedding',
       'obj_to_dum__Loan.Purpose_medical', 'obj_to_dum__Loan.Purpose_moving',
       'obj_to_dum__State_CA', 'obj_to_dum__State_NY', 'obj_to_dum__State_FL',
       'obj_to_dum__State_TX', 'obj_to_dum__State_PA', 'obj_to_dum__State_IL',
       'obj_to_dum__State_GA', 'obj_to_dum__State_NJ', 'obj_to_dum__State_VA',
       'obj_to_dum__State_MA',

In [24]:
y_train=ld_train["Interest.Rate"].str.replace('%','').astype(float)
y_train


0       18.49
1       17.27
2       14.33
3       16.29
4       12.23
        ...  
2195    23.28
2196    14.33
2197    15.31
2198    20.99
2199    10.16
Name: Interest.Rate, Length: 2200, dtype: float64

In [25]:
#lINEAR REGRESSION

In [26]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score

In [27]:
lm=LinearRegression()

In [28]:
cv_mae=cross_val_score(lm,
                       x_train,y_train,
                      cv=10,
                      scoring='neg_mean_absolute_error')

In [29]:
cv_mae.mean()

-1.6090335977701393

In [30]:
cv_mae.std()

0.10727891352839572

In [31]:
lm.fit(x_train,y_train)

LinearRegression()

In [32]:
lm.intercept_

73.04434562219876

In [33]:
lm.coef_

array([ 4.91097030e-05,  1.17248161e-04, -3.68040758e-02, -1.56708629e-06,
       -2.15893596e-03,  1.34580901e+00,  4.50164361e+00, -7.95558356e-01,
       -8.95023654e-01,  3.44101687e-02, -5.64961402e-01, -4.19770245e-01,
       -2.17767837e-01, -5.23497566e-01, -9.50935735e-01, -6.24582154e-01,
        7.10393367e-01, -1.89572262e-01, -1.62077703e-01, -3.83963386e-02,
        4.72213317e-01, -4.71219583e-01, -5.97883907e-01, -1.29119794e-01,
       -3.48019912e-01, -1.62412357e-01, -1.25628925e-01, -5.28670226e-01,
       -1.82785269e-01, -1.03168025e-01,  6.76024766e-02, -2.83418474e-01,
        4.29922908e-01,  3.89627629e-01, -1.00830778e-01,  1.19526588e-02,
       -1.56274811e-01, -2.72330672e-01, -2.18141242e-01, -4.78253620e-02,
       -1.13933823e-03,  1.70071492e-02,  2.44919285e-01, -2.43088247e+00,
       -2.20441699e+00, -2.07656326e+00,  5.52107009e-01,  3.97105772e-01,
        3.52590264e-01,  1.87782877e-01,  6.29124556e-01,  4.61856956e-01,
        3.18842170e-01,  

In [34]:
list(zip(x_train.columns, lm.coef_))

[('obj_to_num__Amount.Requested', 4.9109702984035364e-05),
 ('obj_to_num__Amount.Funded.By.Investors', 0.00011724816069084552),
 ('obj_to_num__Open.CREDIT.Lines', -0.036804075833552975),
 ('obj_to_num__Revolving.CREDIT.Balance', -1.5670862899668783e-06),
 ('dtir__Debt.To.Income.Ratio', -0.002158935955780393),
 ('obj_to_dum__Loan.Length_36 months', 1.3458090091741697),
 ('obj_to_dum__Loan.Length_60 months', 4.501643614256357),
 ('obj_to_dum__Loan.Purpose_debt_consolidation', -0.7955583555394746),
 ('obj_to_dum__Loan.Purpose_credit_card', -0.895023654478015),
 ('obj_to_dum__Loan.Purpose_other', 0.034410168683323306),
 ('obj_to_dum__Loan.Purpose_home_improvement', -0.564961402019623),
 ('obj_to_dum__Loan.Purpose_major_purchase', -0.41977024469914437),
 ('obj_to_dum__Loan.Purpose_small_business', -0.21776783697196683),
 ('obj_to_dum__Loan.Purpose_car', -0.5234975663282307),
 ('obj_to_dum__Loan.Purpose_wedding', -0.950935734592282),
 ('obj_to_dum__Loan.Purpose_medical', -0.6245821540786641)

In [35]:
test_predict=lm.predict(x_test)
test_predict

array([16.70799585, 16.00832441, 10.33709785,  3.69116239, 15.24991944,
        7.64043694, 15.56010771, 10.65005028, 15.941686  , 12.40783892,
       10.07271324, 15.2280325 , 11.67477637, 13.63967139, 13.43076211,
       18.39083847, 10.26336433, 15.77026142, 13.49390545, 14.02604258,
       22.56736997, 17.43996629, 12.41267037, 14.65336533,  9.64344848,
       11.39958837, 13.30956956, 19.1182746 , 11.81628241, 16.79747458,
       15.19456707, 15.27420913, 12.36477469, 14.97510221, 13.86521163,
       14.08348195, 19.65313081, 11.31657445, 12.01385911, 16.9033149 ,
       14.07985435, 11.15670313, 15.00538815, 13.26128284, 15.53903097,
       17.16549476, 14.90394046, 18.85806411, 17.14613739, 10.40842271,
       13.3464016 , 19.82461581, 10.04729388, 19.57442289, 15.76637987,
       15.31454435, 17.17119236, 14.60583533, 10.94554408, 14.60716935,
       13.10262133, 17.22715423,  8.84894718, 14.74112233, 10.01636322,
       11.42004752, 12.84835679, 14.9071138 , 11.24916667, 14.43

In [36]:
pd.DataFrame(test_predict).to_csv("result1.csv",index=False)


In [37]:
import os
os.getcwd()

'E:\\IIT-K\\Linear-Regression-Model-Loan'

# RIDGE REGRESSION

In [40]:
from sklearn.linear_model import Ridge,Lasso
from sklearn.model_selection import GridSearchCV

In [41]:
lambdas=np.linspace(1,100,100)

In [42]:
lambdas

array([  1.,   2.,   3.,   4.,   5.,   6.,   7.,   8.,   9.,  10.,  11.,
        12.,  13.,  14.,  15.,  16.,  17.,  18.,  19.,  20.,  21.,  22.,
        23.,  24.,  25.,  26.,  27.,  28.,  29.,  30.,  31.,  32.,  33.,
        34.,  35.,  36.,  37.,  38.,  39.,  40.,  41.,  42.,  43.,  44.,
        45.,  46.,  47.,  48.,  49.,  50.,  51.,  52.,  53.,  54.,  55.,
        56.,  57.,  58.,  59.,  60.,  61.,  62.,  63.,  64.,  65.,  66.,
        67.,  68.,  69.,  70.,  71.,  72.,  73.,  74.,  75.,  76.,  77.,
        78.,  79.,  80.,  81.,  82.,  83.,  84.,  85.,  86.,  87.,  88.,
        89.,  90.,  91.,  92.,  93.,  94.,  95.,  96.,  97.,  98.,  99.,
       100.])

In [43]:
params={'alpha':lambdas}

In [44]:
params

{'alpha': array([  1.,   2.,   3.,   4.,   5.,   6.,   7.,   8.,   9.,  10.,  11.,
         12.,  13.,  14.,  15.,  16.,  17.,  18.,  19.,  20.,  21.,  22.,
         23.,  24.,  25.,  26.,  27.,  28.,  29.,  30.,  31.,  32.,  33.,
         34.,  35.,  36.,  37.,  38.,  39.,  40.,  41.,  42.,  43.,  44.,
         45.,  46.,  47.,  48.,  49.,  50.,  51.,  52.,  53.,  54.,  55.,
         56.,  57.,  58.,  59.,  60.,  61.,  62.,  63.,  64.,  65.,  66.,
         67.,  68.,  69.,  70.,  71.,  72.,  73.,  74.,  75.,  76.,  77.,
         78.,  79.,  80.,  81.,  82.,  83.,  84.,  85.,  86.,  87.,  88.,
         89.,  90.,  91.,  92.,  93.,  94.,  95.,  96.,  97.,  98.,  99.,
        100.])}

In [46]:
model=Ridge(fit_intercept=True)

In [48]:
model

Ridge()

In [57]:
grid_search=GridSearchCV(model,
                         param_grid=params,
                         cv=10,
                         scoring='neg_mean_absolute_error',
                         verbose=20,
                         n_jobs=-1)

In [59]:
grid_search.fit(x_train,y_train)

Fitting 10 folds for each of 100 candidates, totalling 1000 fits


GridSearchCV(cv=10, estimator=Ridge(), n_jobs=-1,
             param_grid={'alpha': array([  1.,   2.,   3.,   4.,   5.,   6.,   7.,   8.,   9.,  10.,  11.,
        12.,  13.,  14.,  15.,  16.,  17.,  18.,  19.,  20.,  21.,  22.,
        23.,  24.,  25.,  26.,  27.,  28.,  29.,  30.,  31.,  32.,  33.,
        34.,  35.,  36.,  37.,  38.,  39.,  40.,  41.,  42.,  43.,  44.,
        45.,  46.,  47.,  48.,  49.,  50.,  51.,  52.,  53.,  54.,  55.,
        56.,  57.,  58.,  59.,  60.,  61.,  62.,  63.,  64.,  65.,  66.,
        67.,  68.,  69.,  70.,  71.,  72.,  73.,  74.,  75.,  76.,  77.,
        78.,  79.,  80.,  81.,  82.,  83.,  84.,  85.,  86.,  87.,  88.,
        89.,  90.,  91.,  92.,  93.,  94.,  95.,  96.,  97.,  98.,  99.,
       100.])},
             scoring='neg_mean_absolute_error', verbose=20)

In [60]:
grid_search.best_estimator_

Ridge(alpha=52.0)

In [63]:
grid_search.cv_results_

{'mean_fit_time': array([0.01220932, 0.01280925, 0.00781455, 0.01126943, 0.00960677,
        0.0092067 , 0.00970719, 0.00930724, 0.01010723, 0.00940671,
        0.00940721, 0.00910735, 0.00990796, 0.00910718, 0.00930707,
        0.00930777, 0.00910645, 0.00890641, 0.00990744, 0.0092067 ,
        0.00910711, 0.00910673, 0.00890684, 0.00920641, 0.00990734,
        0.00970731, 0.00940616, 0.00930622, 0.00890629, 0.00890636,
        0.00900648, 0.01030767, 0.00910802, 0.00960763, 0.00900714,
        0.00880764, 0.0092078 , 0.00940731, 0.00900638, 0.00920699,
        0.00910718, 0.00890727, 0.00880706, 0.00920689, 0.00980742,
        0.00880675, 0.00900755, 0.01000762, 0.00950768, 0.009008  ,
        0.01010745, 0.00930669, 0.00900695, 0.00920765, 0.00920784,
        0.00950723, 0.01040711, 0.0098069 , 0.00970752, 0.00910614,
        0.00850666, 0.0087069 , 0.00990775, 0.01000729, 0.00840635,
        0.00950792, 0.00960751, 0.00960767, 0.0096077 , 0.00920725,
        0.00910823, 0.0094075 ,

In [ ]:
def report(re)